##### Imports

In [3]:
import pandas as pd
import nltk
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import pickle
from sklearn.preprocessing import LabelEncoder

##### Load in Data

In [2]:
text_df = pd.read_pickle("merged_training.pkl")
text_df = text_df.reset_index(drop = True)
text_df

,text,emotions
0,i feel awful about it too because it s my job ...,sadness
1,im alone i feel awful,sadness
2,ive probably mentioned this before but i reall...,joy
3,i was feeling a little low few days back,sadness
4,i beleive that i am much more sensitive to oth...,love
...,...,...
416804,that was what i felt when i was finally accept...,joy
416805,i take every day as it comes i m just focussin...,fear
416806,i just suddenly feel that everything was fake,sadness
416807,im feeling more eager than ever to claw back w...,joy


##### Tokenize Text

In [3]:
# store data by words
data_by_words = []
# loop through texts
for i in text_df['text']:
    # get words, tokenize
    value = nltk.word_tokenize(i)
    data_by_words.append(value)
data_by_words[0]

['i',
 'feel',
 'awful',
 'about',
 'it',
 'too',
 'because',
 'it',
 's',
 'my',
 'job',
 'to',
 'get',
 'him',
 'in',
 'a',
 'position',
 'to',
 'succeed',
 'and',
 'it',
 'just',
 'didn',
 't',
 'happen',
 'here']

##### Get Document Embeddings

In [4]:
model = Word2Vec(sentences = data_by_words, vector_size = 100)
model.train(data_by_words, total_examples = len(data_by_words), epochs = 50)

(271017777, 400544400)

In [5]:
model.save("word2vec.model")

In [6]:
embeddings_word_data = []
# for each document
for i in data_by_words:
    text = []
    # for each word in doc
    for j in i:
        # if word in model vocab get the embedding
        if j in model.wv.key_to_index:
            text.append(model.wv[j])
    # average embeddings across all word embeddings
    text = np.mean(text, axis = 0)
    embeddings_word_data.append(text)
embeddings_word_data = np.array(embeddings_word_data)
print(embeddings_word_data.shape)

c:\Users\jenni\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\jenni\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(416809,)


<ipython-input-6-346ecf699279>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embeddings_word_data = np.array(embeddings_word_data)


In [5]:
# split into train, val, test
labelencoder = LabelEncoder()
y = list(text_df['emotions'])
y = labelencoder.fit_transform(y)

X_train, X_val_test, y_train, y_val_test = train_test_split(embeddings_word_data, y, test_size=0.2)

X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.50)

In [11]:
# save embeddings
np.save('text_embeddings_train.npy', X_train)
np.save('text_embeddings_val.npy', X_val)
np.save('text_embeddings_test.npy', X_test)

In [8]:
# save labels
with open("train_y.pkl", "wb") as file:
    pickle.dump(y_train, file)
    
with open("val_y.pkl", "wb") as file:
    pickle.dump(y_val, file)
    
with open("test_y.pkl", "wb") as file:
    pickle.dump(y_test, file)